In [4]:
import tkinter as tk
from tkinter import ttk, Scrollbar, Text, filedialog, messagebox
import numpy as np
import tensorflow as tf
import time
from datetime import datetime
import sqlite3
import spacy
import subprocess
import sys
import threading
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model
import cv2
import pyaudio
from PIL import Image, ImageTk
import wave
import pyttsx3

# ==================================================
# Configuración del Chatbot
# ==================================================
def load_spacy_model():
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)
        return spacy.load("en_core_web_sm")

nlp = load_spacy_model()

conn = sqlite3.connect("alzheimers_chatbot.db")
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS faq (question TEXT, answer TEXT)''')

faq_data = [
    ("¿Qué es el Alzheimer?", "El Alzheimer es una enfermedad neurodegenerativa que afecta la memoria y otras funciones cognitivas."),
    # ... (resto de preguntas predefinidas)
]

c.executemany("INSERT OR IGNORE INTO faq (question, answer) VALUES (?, ?)", faq_data)
conn.commit()

questions = [q for q, a in faq_data]
vectorizer = TfidfVectorizer().fit(questions)

def get_response(user_input):
    user_input = user_input.lower()
    user_vector = vectorizer.transform([user_input])
    question_vectors = vectorizer.transform(questions)
    similarities = cosine_similarity(user_vector, question_vectors)
    most_similar_index = similarities.argmax()
    highest_similarity = similarities[0, most_similar_index]
    
    if highest_similarity > 0.5:
        return faq_data[most_similar_index][1]
    
    doc = nlp(user_input)
    for keyword in ["síntomas", "factores", "tratamiento", "diagnóstico", "prevención", "memoria", "cuidado", "comunicación", "estrés"]:
        if keyword in user_input:
            return "Parece que preguntas sobre " + keyword + ". ¿Podrías ser más específico?"
    
    return "Lo siento, no tengo una respuesta para eso. Intenta reformular tu pregunta."

# ==================================================
# Configuración de Cámara y Audio Mejorada
# ==================================================
class CameraApp:
    def __init__(self, window, window_label):
        self.window = window
        self.video_source = 0
        self.vid = None
        self.camera_fps = 30
        self.is_camera_on = False
        self.audio_stream_in = None
        self.audio_stream_out = None
        self.audio = pyaudio.PyAudio()
        self.muted = False
        self.frame = None
        self.available_cameras = []
        self.available_audio_devices = []
        self.current_camera = tk.StringVar()
        self.current_audio_in = tk.StringVar()
        self.current_audio_out = tk.StringVar()
        self.current_voice = tk.StringVar()
        
        self.detect_available_cameras()
        self.detect_audio_devices()
        self.setup_gui()
        self.tts_engine = pyttsx3.init()
        self.setup_voices()

    def detect_available_cameras(self):
        self.available_cameras = []
        for i in range(5):
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                self.available_cameras.append(f"Cámara {i}")
                cap.release()
        if not self.available_cameras:
            self.available_cameras = ["No se encontraron cámaras"]

    def detect_audio_devices(self):
        self.available_audio_devices = []
        for i in range(self.audio.get_device_count()):
            info = self.audio.get_device_info_by_index(i)
            self.available_audio_devices.append(info['name'])
    
    def setup_voices(self):
        voices = self.tts_engine.getProperty('voices')
        self.voice_names = [voice.name for voice in voices]
        self.voice_selector['values'] = self.voice_names
        if self.voice_names:
            self.current_voice.set(self.voice_names[0])

    def setup_gui(self):
        # Selectores de dispositivos
        ttk.Label(self.window, text="Seleccionar Cámara:").pack()
        self.camera_selector = ttk.Combobox(self.window, textvariable=self.current_camera, 
                                          values=self.available_cameras)
        self.camera_selector.pack()
        
        ttk.Label(self.window, text="Micrófono:").pack()
        self.audio_in_selector = ttk.Combobox(self.window, textvariable=self.current_audio_in,
                                            values=self.available_audio_devices)
        self.audio_in_selector.pack()
        
        ttk.Label(self.window, text="Salida de Audio:").pack()
        self.audio_out_selector = ttk.Combobox(self.window, textvariable=self.current_audio_out,
                                              values=self.available_audio_devices)
        self.audio_out_selector.pack()
        
        ttk.Label(self.window, text="Voz TTS:").pack()
        self.voice_selector = ttk.Combobox(self.window, textvariable=self.current_voice)
        self.voice_selector.pack()
        
        # Botones de control
        self.btn_frame = tk.Frame(self.window)
        self.btn_frame.pack(pady=10)
        
        self.btn_start = tk.Button(self.btn_frame, text="Iniciar", command=self.start_camera)
        self.btn_start.pack(side=tk.LEFT, padx=5)
        
        self.btn_stop = tk.Button(self.btn_frame, text="Detener", command=self.stop_camera, state=tk.DISABLED)
        self.btn_stop.pack(side=tk.LEFT, padx=5)
        
        self.btn_mute = tk.Button(self.btn_frame, text="Silenciar", command=self.toggle_mute)
        self.btn_mute.pack(side=tk.LEFT, padx=5)
        
        # Canvas para video
        self.canvas = tk.Canvas(self.window, width=640, height=480)
        self.canvas.pack()
        
        # Medidor de audio
        self.sound_bar = tk.Scale(self.window, from_=0, to=100, orient=tk.HORIZONTAL, 
                                length=640, state=tk.DISABLED)
        self.sound_bar.pack()
        self.sound_label = tk.Label(self.window, text="dB: 0")
        self.sound_label.pack()

    def start_camera(self):
        if not self.is_camera_on:
            cam_index = int(self.current_camera.get().split()[-1])
            self.vid = cv2.VideoCapture(cam_index)
            self.camera_fps = self.vid.get(cv2.CAP_PROP_FPS)
            if self.camera_fps <= 0:
                self.camera_fps = 30
            
            self.is_camera_on = True
            self.btn_start.config(state=tk.DISABLED)
            self.btn_stop.config(state=tk.NORMAL)
            self.start_audio_streams()
            self.update_camera()
            self.update_audio()

    def stop_camera(self):
        if self.is_camera_on:
            self.is_camera_on = False
            self.vid.release()
            self.stop_audio_streams()
            self.btn_start.config(state=tk.NORMAL)
            self.btn_stop.config(state=tk.DISABLED)

    def toggle_mute(self):
        self.muted = not self.muted
        self.btn_mute.config(text="Activar sonido" if self.muted else "Silenciar")

    def start_audio_streams(self):
        # Configurar audio de entrada
        input_index = self.audio_in_selector.current()
        self.audio_stream_in = self.audio.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=44100,
            input=True,
            input_device_index=input_index,
            frames_per_buffer=1024
        )
        
        # Configurar audio de salida
        output_index = self.audio_out_selector.current()
        self.audio_stream_out = self.audio.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=44100,
            output=True,
            output_device_index=output_index,
            frames_per_buffer=1024
        )

    def stop_audio_streams(self):
        if self.audio_stream_in:
            self.audio_stream_in.stop_stream()
            self.audio_stream_in.close()
        if self.audio_stream_out:
            self.audio_stream_out.stop_stream()
            self.audio_stream_out.close()

    def update_camera(self):
        if self.is_camera_on:
            ret, frame = self.vid.read()
            if ret:
                self.frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(self.frame)
                imgtk = ImageTk.PhotoImage(image=img)
                self.canvas.imgtk = imgtk
                self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
            self.window.after(int(1000/self.camera_fps), self.update_camera)

    def update_audio(self):
        if self.is_camera_on and not self.muted:
            try:
                data = self.audio_stream_in.read(1024)
                if self.audio_stream_out.is_active():
                    self.audio_stream_out.write(data)
                
                # Calcular nivel de audio
                audio_data = np.frombuffer(data, dtype=np.int16)
                volume = np.abs(audio_data).mean()
                db = 20 * np.log10(volume) if volume > 0 else 0
                self.sound_bar.set(db)
                self.sound_label.config(text=f"dB: {db:.2f}")
            except Exception as e:
                print(f"Error de audio: {e}")
        self.window.after(10, self.update_audio)

    def speak(self, text):
        voices = self.tts_engine.getProperty('voices')
        selected_voice = [v for v in voices if v.name == self.current_voice.get()]
        if selected_voice:
            self.tts_engine.setProperty('voice', selected_voice[0].id)
        self.tts_engine.say(text)
        self.tts_engine.runAndWait()

# ==================================================
# Interfaz Gráfica Principal
# ==================================================
ventana = tk.Tk()
ventana.title("Sistema Avanzado de Monitoreo para Alzheimer")
ventana.geometry("1200x900")

# Configurar pestañas
tab_control = ttk.Notebook(ventana)
tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)
tab3 = ttk.Frame(tab_control)
tab_control.add(tab1, text='Cámara y Audio')
tab_control.add(tab2, text='Chatbot')
tab_control.add(tab3, text='Predicción')
tab_control.pack(expand=1, fill='both')

# Pestaña de Cámara y Audio
camera_app = CameraApp(tab1, "Cámara Digital")

# Pestaña de Chatbot (similar al código original)
# ... (implementar interfaz de chatbot similar al código original)

# Pestaña de Predicción (similar al código original)
# ... (implementar interfaz de predicción similar al código original)

# Iniciar aplicación
ventana.mainloop()

# Limpieza final
conn.close()
camera_app.stop_camera()
if camera_app.tts_engine:
    camera_app.tts_engine.stop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\pablo\AppData\Local\Temp\ipykernel_12948\1200254542.py", line 173, in start_camera
    self.start_audio_streams()
  File "C:\Users\pablo\AppData\Local\Temp\ipykernel_12948\1200254542.py", line 203, in start_audio_streams
    self.audio_stream_out = self.audio.open(
                            ^^^^^^^^^^^^^^^^
  File "c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyaudio\__init__.py", line 639, in open
    stream = PyAudio.Stream(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyaudio\__init__.py", line 441, in __init__
    self._stream = pa.open(**arguments)
                   ^^^^^^^^^^^^^^^^^^^^
OSError: [E

In [6]:
import tkinter as tk
from tkinter import ttk, Scrollbar, Text, filedialog, messagebox
import numpy as np
import tensorflow as tf
import time
from datetime import datetime
import sqlite3
import spacy
import subprocess
import sys
import threading
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model
import cv2
import pyaudio
from PIL import Image, ImageTk
import pyttsx3

# ==================================================
# Configuración del Chatbot
# ==================================================
def load_spacy_model():
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)
        return spacy.load("en_core_web_sm")

nlp = load_spacy_model()

conn = sqlite3.connect("alzheimers_chatbot.db")
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS faq (question TEXT, answer TEXT)''')

faq_data = [
    ("¿Qué es el Alzheimer?", "El Alzheimer es una enfermedad neurodegenerativa que afecta la memoria y otras funciones cognitivas."),
    # ... (resto de las preguntas predefinidas)
]

c.executemany("INSERT OR IGNORE INTO faq (question, answer) VALUES (?, ?)", faq_data)
conn.commit()

questions = [q for q, a in faq_data]
vectorizer = TfidfVectorizer().fit(questions)

def get_response(user_input):
    user_input = user_input.lower()
    user_vector = vectorizer.transform([user_input])
    question_vectors = vectorizer.transform(questions)
    similarities = cosine_similarity(user_vector, question_vectors)
    most_similar_index = similarities.argmax()
    highest_similarity = similarities[0, most_similar_index]
    
    if highest_similarity > 0.5:
        return faq_data[most_similar_index][1]
    
    doc = nlp(user_input)
    for keyword in ["síntomas", "factores", "tratamiento", "diagnóstico", "prevención", "memoria", "cuidado", "comunicación", "estrés"]:
        if keyword in user_input:
            return f"Parece que preguntas sobre {keyword}. ¿Podrías ser más específico?"
    
    return "Lo siento, no tengo una respuesta para eso. Intenta reformular tu pregunta."

# ==================================================
# Configuración Mejorada de Cámara y Audio
# ==================================================
class EnhancedCameraApp:
    def __init__(self, window, window_label):
        self.window = window
        self.window_label = window_label
        self.video_source = 0
        self.vid = None
        self.camera_fps = 30
        self.is_camera_on = False
        self.audio_stream_in = None
        self.audio_stream_out = None
        self.audio = pyaudio.PyAudio()
        self.muted = False
        self.available_cameras = self.detect_cameras()
        self.available_mics = self.detect_audio_devices('input')
        self.available_speakers = self.detect_audio_devices('output')
        self.tts_engine = pyttsx3.init()
        self.voices = self.tts_engine.getProperty('voices')
        
        self.setup_ui()

    def detect_cameras(self):
        cameras = []
        for i in range(4):
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                cameras.append(f"Cámara {i}")
                cap.release()
        return cameras or ["No se detectaron cámaras"]

    def detect_audio_devices(self, kind='input'):
        devices = []
        for i in range(self.audio.get_device_count()):
            info = self.audio.get_device_info_by_index(i)
            if (kind == 'input' and info['maxInputChannels'] > 0) or \
               (kind == 'output' and info['maxOutputChannels'] > 0):
                devices.append(info['name'])
        return devices or ["Dispositivo no detectado"]

    def setup_ui(self):
        control_frame = ttk.Frame(self.window)
        control_frame.pack(pady=10)
        
        # Selectores de dispositivos
        ttk.Label(control_frame, text="Cámara:").grid(row=0, column=0, padx=5)
        self.cam_selector = ttk.Combobox(control_frame, values=self.available_cameras)
        self.cam_selector.grid(row=0, column=1, padx=5)
        self.cam_selector.current(0)
        
        ttk.Label(control_frame, text="Micrófono:").grid(row=0, column=2, padx=5)
        self.mic_selector = ttk.Combobox(control_frame, values=self.available_mics)
        self.mic_selector.grid(row=0, column=3, padx=5)
        self.mic_selector.current(0)
        
        ttk.Label(control_frame, text="Altavoz:").grid(row=0, column=4, padx=5)
        self.speaker_selector = ttk.Combobox(control_frame, values=self.available_speakers)
        self.speaker_selector.grid(row=0, column=5, padx=5)
        self.speaker_selector.current(0)
        
        ttk.Label(control_frame, text="Voz:").grid(row=0, column=6, padx=5)
        self.voice_selector = ttk.Combobox(control_frame, values=[v.name for v in self.voices])
        self.voice_selector.grid(row=0, column=7, padx=5)
        self.voice_selector.current(0)
        
        # Botones de control
        self.btn_frame = ttk.Frame(self.window)
        self.btn_frame.pack(pady=10)
        
        self.btn_start = ttk.Button(self.btn_frame, text="Iniciar", command=self.start_all)
        self.btn_start.pack(side=tk.LEFT, padx=5)
        
        self.btn_stop = ttk.Button(self.btn_frame, text="Detener", command=self.stop_all, state=tk.DISABLED)
        self.btn_stop.pack(side=tk.LEFT, padx=5)
        
        self.btn_mute = ttk.Button(self.btn_frame, text="Silenciar", command=self.toggle_mute)
        self.btn_mute.pack(side=tk.LEFT, padx=5)
        
        # Canvas y audio
        self.canvas = tk.Canvas(self.window, width=640, height=480)
        self.canvas.pack()
        
        self.sound_bar = ttk.Progressbar(self.window, length=640, mode='determinate')
        self.sound_bar.pack()
        self.sound_label = ttk.Label(self.window, text="dB: 0.00")
        self.sound_label.pack()

    def start_all(self):
        self.start_camera()
        self.start_audio()
        self.btn_start.config(state=tk.DISABLED)
        self.btn_stop.config(state=tk.NORMAL)

    def stop_all(self):
        self.stop_camera()
        self.stop_audio()
        self.btn_start.config(state=tk.NORMAL)
        self.btn_stop.config(state=tk.DISABLED)

    def start_camera(self):
        if not self.is_camera_on:
            self.video_source = int(self.cam_selector.get().split()[-1])
            self.vid = cv2.VideoCapture(self.video_source)
            self.camera_fps = self.vid.get(cv2.CAP_PROP_FPS) or 30
            self.is_camera_on = True
            self.update_camera()

    def stop_camera(self):
        if self.is_camera_on:
            self.is_camera_on = False
            self.vid.release()
            self.canvas.delete("all")

    def start_audio(self):
        input_index = self.mic_selector.current()
        output_index = self.speaker_selector.current()
        
        self.audio_stream_in = self.audio.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=44100,
            input=True,
            input_device_index=input_index,
            frames_per_buffer=1024
        )
        
        self.audio_stream_out = self.audio.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=44100,
            output=True,
            output_device_index=output_index,
            frames_per_buffer=1024
        )
        
        self.update_audio()

    def stop_audio(self):
        if self.audio_stream_in:
            self.audio_stream_in.stop_stream()
            self.audio_stream_in.close()
        if self.audio_stream_out:
            self.audio_stream_out.stop_stream()
            self.audio_stream_out.close()

    def toggle_mute(self):
        self.muted = not self.muted
        self.btn_mute.config(text="Activar sonido" if self.muted else "Silenciar")

    def update_camera(self):
        if self.is_camera_on:
            ret, frame = self.vid.read()
            if ret:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(rgb)
                imgtk = ImageTk.PhotoImage(image=img)
                self.canvas.imgtk = imgtk
                self.canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
            self.window.after(int(1000/self.camera_fps), self.update_camera)

    def update_audio(self):
        if self.is_camera_on and not self.muted:
            try:
                data = self.audio_stream_in.read(1024, exception_on_overflow=False)
                self.audio_stream_out.write(data)
                
                audio_data = np.frombuffer(data, dtype=np.int16)
                rms = np.sqrt(np.mean(audio_data**2))
                db = 20 * np.log10(rms) if rms > 0 else 0
                self.sound_bar['value'] = db
                self.sound_label.config(text=f"dB: {db:.2f}")
            except Exception as e:
                print(f"Error de audio: {e}")
        self.window.after(10, self.update_audio)

    def speak(self, text):
        self.tts_engine.setProperty('voice', self.voices[self.voice_selector.current()].id)
        self.tts_engine.say(text)
        self.tts_engine.runAndWait()

# ==================================================
# Interfaz Gráfica Principal
# ==================================================
class MainApplication(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Sistema de Monitoreo Mejorado para Alzheimer")
        self.geometry("1200x800")
        
        self.setup_tabs()
        self.setup_chatbot()
        self.setup_prediction()
        self.start_monitoring()

    def setup_tabs(self):
        self.tab_control = ttk.Notebook(self)
        
        self.tab1 = ttk.Frame(self.tab_control)
        self.tab2 = ttk.Frame(self.tab_control)
        self.tab3 = ttk.Frame(self.tab_control)
        
        self.tab_control.add(self.tab1, text="Cámara")
        self.tab_control.add(self.tab2, text="Chatbot")
        self.tab_control.add(self.tab3, text="Predicción")
        
        self.tab_control.pack(expand=1, fill="both")
        
        self.camera_system = EnhancedCameraApp(self.tab1, "Cámara Digital")

    def setup_chatbot(self):
        # ... (mantener la implementación original del chatbot)
        pass

    def setup_prediction(self):
        # ... (mantener la implementación original de predicción)
        pass

    def start_monitoring(self):
        # ... (mantener la implementación original de monitoreo)
        pass

if __name__ == "__main__":
    app = MainApplication()
    app.mainloop()
    conn.close()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\pablo\AppData\Local\Temp\ipykernel_12948\2493183370.py", line 155, in start_all
    self.start_audio()
  File "C:\Users\pablo\AppData\Local\Temp\ipykernel_12948\2493183370.py", line 192, in start_audio
    self.audio_stream_out = self.audio.open(
                            ^^^^^^^^^^^^^^^^
  File "c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyaudio\__init__.py", line 639, in open
    stream = PyAudio.Stream(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pablo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyaudio\__init__.py", line 441, in __init__
    self._stream = pa.open(**arguments)
                   ^^^^^^^^^^^^^^^^^^^^
OSError: [Errno -9998] Invalid